In [ ]:
import os
import xml.etree.ElementTree as ET

# Define the path of YOLO folder and PASCAL VOC folder
yolo_path = 'YOLO'
pascal_path = 'PASCAL VOC'

# Read the classes from the classes.txt file
with open(os.path.join(yolo_path, 'classes.txt'), 'r') as f:
    classes = f.read().splitlines()

# Create the PASCAL VOC folder structure
if not os.path.exists(pascal_path):
    os.makedirs(os.path.join(pascal_path, 'Annotations'))

# Convert each YOLO annotation file to PASCAL VOC format
for file_name in os.listdir(os.path.join(yolo_path, 'Annotations')):
    # Read the YOLO annotation file
    with open(os.path.join(yolo_path, 'Annotations', file_name), 'r') as f:
        lines = f.read().splitlines()

    # Create the PASCAL VOC annotation file
    annotation = ET.Element('annotation')
    ET.SubElement(annotation, 'folder').text = 'JPEGImages'
    ET.SubElement(annotation, 'filename').text = file_name[:-4] + '.jpg'
    ET.SubElement(annotation, 'source').text = 'unknown'
    size = ET.SubElement(annotation, 'size')
    ET.SubElement(size, 'width').text = '0'
    ET.SubElement(size, 'height').text = '0'
    ET.SubElement(size, 'depth').text = '3'
    ET.SubElement(annotation, 'segmented').text = '0'

    # Add object annotation for each object in the YOLO annotation file
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.split())
        class_name = classes[int(class_id)]
        object_annotation = ET.SubElement(annotation, 'object')
        ET.SubElement(object_annotation, 'name').text = class_name
        ET.SubElement(object_annotation, 'pose').text = 'Unspecified'
        ET.SubElement(object_annotation, 'truncated').text = '0'
        ET.SubElement(object_annotation, 'difficult').text = '0'
        bndbox = ET.SubElement(object_annotation, 'bndbox')
        xmin = x_center - width/2
        ymin = y_center - height/2
        xmax = x_center + width/2
        ymax = y_center + height/2
        ET.SubElement(bndbox, 'xmin').text = str(int(xmin))
        ET.SubElement(bndbox, 'ymin').text = str(int(ymin))
        ET.SubElement(bndbox, 'xmax').text = str(int(xmax))
        ET.SubElement(bndbox, 'ymax').text = str(int(ymax))

    # Save the PASCAL VOC annotation file
    tree = ET.ElementTree(annotation)
    tree.write(os.path.join(pascal_path, 'Annotations', file_name[:-4] + '.xml'))
